In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append('./coeqwalpackage')

from coeqwalpackage.metrics import (read_in_df, add_water_year_column, compute_storage_thresholds, compute_metrics_suite, normalize_index, percent_change_from_baseline)
from coeqwalpackage import cqwlutils as cu
from coeqwalpackage import plotting as pu

In [3]:
pu.enable_headless_mode()

In [4]:
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex

<cell_type>markdown</cell_type>## Notebook Configuration

In [5]:
# Control file settings
CTRL_FILE = 'CalSim3DataExtractionInitFile_v4.xlsx'
CTRL_TAB = 'Init'

BASELINE_ID = "s0020"
HIGHLIGHT_SCENARIOS = [10, 11, 20, 21, 23, 24]

DPI = 300

S_MLRTN_LEVEL5_TAF = 524 # Millerton Lake flood pool
S_MLRTN_LEVEL1_TAF = 135 # Millerton Lake dead pool
S_MELON_LEVEL1_TAF = 80 # New Melones dead pool

VARIABLES_STORAGE = ["S_SHSTA_", "S_OROVL_", "S_TRNTY_",  "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MLRTN_", "S_MELON_"]

VARIABLES_FLOODPOOL = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV", "S_MLRTN_LEVEL5DV", "S_MELONLEVEL4DV"]

VARIABLES_DEADPOOL = ["S_SHSTALEVEL1DV", "S_OROVLLEVEL1DV", "S_TRNTYLEVEL1DV", "S_SLUIS_CVPLEVEL1DV", "S_SLUIS_SWPLEVEL1DV", "S_MLRTN_LEVEL1DV", "S_MELONLEVEL1DV"]

FLOW_ANALYSIS_SCENARIO = "s0018"

## Read from control file

In [6]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CTRL_FILE, CTRL_TAB)

### Read in Data

In [7]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df = add_water_year_column(df)

In [8]:
print(f"Loaded data frame shape: {df.shape}")
print(f"Total scenarios: {len(dss_names)}")

Loaded data frame shape: (1200, 13020)
Total scenarios: 35


In [ ]:
scenario_names = cu.build_scenario_labels_from_listing(ScenarioListPath, ScenarioListFile, ScenarioListTab)
scenario_label_lookup = {f"s{sid:04d}": label for sid, label in scenario_names.items()}
highlight_ids = []

for item in HIGHLIGHT_SCENARIOS:
    if isinstance(item, str) and item.lower().startswith('s'):
        highlight_ids.append(item.lower())
    else:
        highlight_ids.append(f"s{int(item):04d}")
        
highlight_ids = [hid.lower() for hid in highlight_ids]
color_pool = [to_hex(c) for c in (plt.cm.tab10.colors + plt.cm.Set2.colors + plt.cm.Dark2.colors)]
highlight_colors = []
color_idx = 0

for hid in highlight_ids:
    if hid == BASELINE_ID.lower():
        highlight_colors.append('#000000')
    else:
        highlight_colors.append(color_pool[color_idx % len(color_pool)])
        color_idx += 1
        
highlight_labels = [scenario_label_lookup.get(hid.lower(), hid.upper()) for hid in highlight_ids]
highlight_config = list(zip(highlight_ids, highlight_colors, highlight_labels))
baseline_label_text = scenario_label_lookup.get(BASELINE_ID.lower(), BASELINE_ID)

### Get metrics paths and make directories if needed

In [10]:
metrics_root = Path(GroupDataDirPath) / "metrics_output"
metrics_root.mkdir(parents=True, exist_ok=True)
plots_dir = metrics_root / "parallel_plots"
plots_dir.mkdir(parents=True, exist_ok=True)

## Metrics Examples

In [11]:
write_out_dfs = []

In [12]:
thresholds, threshold_outputs = compute_storage_thresholds(df, dss_names, VARIABLES_STORAGE, VARIABLES_DEADPOOL, VARIABLES_FLOODPOOL, mlrtn_level5constant=S_MLRTN_LEVEL5_TAF, mlrtn_level1constant=S_MLRTN_LEVEL1_TAF, smelon_level1constant=S_MELON_LEVEL1_TAF)

write_out_dfs.extend(threshold_outputs)
print(f"Computed {len(thresholds)} storage threshold metrics")

Variables:
['S_SHSTA_', 'S_OROVL_', 'S_TRNTY_', 'S_SLUIS_CVP_', 'S_SLUIS_SWP_', 'S_MLRTN_', 'S_MELON_']
Current variable: S_SHSTA_
Current variable: S_OROVL_
Current variable: S_TRNTY_
Current variable: S_SLUIS_CVP_
Current variable: S_SLUIS_SWP_
Current variable: S_MLRTN_
Current variable: S_MELON_
Computed 28 storage threshold metrics


In [13]:
# Flow Requirements - parameterized via FLOW_ANALYSIS_SCENARIO in config
# Set FLOW_ANALYSIS_SCENARIO = None to skip this analysis

if FLOW_ANALYSIS_SCENARIO is not None:
    _scen = FLOW_ANALYSIS_SCENARIO
    actual_flows = [f"C_AMR004_{_scen}", f"C_TRN111_{_scen}", f"C_FTR029_{_scen}", f"C_MOK028_{_scen}", f"C_MCD005_{_scen}", f"C_SAC257_{_scen}", f"C_SJR127_{_scen}", f"C_SAC289_{_scen}", f"C_STS011_{_scen}", f"C_TUO003_{_scen}", f"C_YUB002_{_scen}", f"SP_SAC159_BTC003_{_scen}", f"C_SAC148_{_scen}", f"C_SAC122_{_scen}" f"C_FTR003_{_scen}", f"C_SAC049_{_scen}", f"C_SJR070_{_scen}"]
    
    min_flows = [f"C_AMR004_MIF_{_scen}", f"C_TRN111_MIF_{_scen}", f"C_FTR029_MIF_{_scen}", f"C_MOK028_MIF_{_scen}", f"C_MCD005_MIF_{_scen}", f"C_SAC257_MIF_{_scen}", f"C_SJR127_MIF_{_scen}", f"C_SAC289_MIF_{_scen}", f"C_STS011_MIF_{_scen}", f"C_TUO003_MIF_{_scen}", f"C_YUB002_MIF_{_scen}", f"SP_SAC159_BTC003_MIF_{_scen}", f"C_SAC148_MIF_{_scen}", f"C_SAC122_MIF_{_scen}",f"C_FTR003_MIF_{_scen}", f"C_SAC049_MIF_{_scen}", f"C_SJR070_MIF_{_scen}"]
    
    print(f"[INFO] Flow analysis enabled for scenario: {FLOW_ANALYSIS_SCENARIO}")
    
else:
    actual_flows = []
    min_flows = []
    print("[INFO] Flow analysis skipped (FLOW_ANALYSIS_SCENARIO is None)")

[INFO] Flow analysis enabled for scenario: s0018


In [14]:
# Metrics
variables = ["S_RESTOT_NOD_", "S_RESTOT_s", "S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_FOLSM_", "S_MELON_", "S_MLRTN_","S_SLUIS_SWP", "S_SLUIS_CVP", "DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_", "D_TOTAL_", "NDO_", "X2_PRV_KM_","EM_EC_MONTH_","JP_EC_MONTH", "DEL_CVP_PSC_N_", "DEL_CVP_PAG_N_", "DEL_CVP_PAG_S_", "DEL_CVP_PRF_S_", "DEL_CVP_PMI_N_", "DEL_CVP_PMI_S_"] 

In [15]:
metrics, metric_outputs = compute_metrics_suite(df, dss_names, variables)
write_out_dfs.extend(metric_outputs)
print(f"Computed {len(metrics)} metric categories: {list(metrics.keys())}")

Computed 59 metric categories: ['Apr_S_RESTOT_NOD_mnth_avg', 'Sept_S_RESTOT_NOD_mnth_avg', 'S_RESTOT_NOD_ann_avg', 'Apr_S_RESTOT_smnth_avg', 'Sept_S_RESTOT_smnth_avg', 'S_RESTOT_sann_avg', 'Apr_S_SHSTA__mnth_avg', 'Sept_S_SHSTA__mnth_avg', 'AprS_SHSTA__CV', 'Sept_S_SHSTA__CV', 'Apr_S_OROVL__mnth_avg', 'Sept_S_OROVL__mnth_avg', 'AprS_OROVL__CV', 'Sept_S_OROVL__CV', 'Apr_S_TRNTY__mnth_avg', 'Sept_S_TRNTY__mnth_avg', 'AprS_TRNTY__CV', 'Sept_S_TRNTY__CV', 'Apr_S_FOLSM__mnth_avg', 'Sept_S_FOLSM__mnth_avg', 'AprS_FOLSM__CV', 'Sept_S_FOLSM__CV', 'Apr_S_MELON__mnth_avg', 'Sept_S_MELON__mnth_avg', 'AprS_MELON__CV', 'Sept_S_MELON__CV', 'Apr_S_MLRTN__mnth_avg', 'Sept_S_MLRTN__mnth_avg', 'AprS_MLRTN__CV', 'Sept_S_MLRTN__CV', 'Apr_S_SLUIS_SWP_mnth_avg', 'Sept_S_SLUIS_SWP_mnth_avg', 'AprS_SLUIS_SWP_CV', 'Sept_S_SLUIS_SWP_CV', 'Apr_S_SLUIS_CVP_mnth_avg', 'Sept_S_SLUIS_CVP_mnth_avg', 'AprS_SLUIS_CVP_CV', 'Sept_S_SLUIS_CVP_CV', 'DEL_SWP_TOTAL__ann_avg', 'DEL_SWP_PMI__ann_avg', 'DEL_SWP_PAG__ann_avg', '

## Write out dataframes to single CSV

In [16]:
write_out_dfs = [normalize_index(df) for df in write_out_dfs]
combined_df = pd.concat(write_out_dfs, axis=1)
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

In [17]:
combined_df.to_csv(metrics_root / "all_metrics_output.csv")

## Calculate the flow difference and plot it. (What was this for??)

In [18]:
def calculate_flow_difference(actual_csv, mif_csv):
    """Calculate difference between actual and minimum instream flows from CSV exports."""
    actual = pd.read_csv(actual_csv, skiprows=6, names=["Index", "Date", "Flow"])
    actual["Date"] = pd.to_datetime(actual["Date"], format="%d%b%Y", errors="coerce")
    actual["Flow"] = pd.to_numeric(actual["Flow"], errors="coerce")
    actual = actual.dropna(subset=["Date", "Flow"])
    actual.set_index("Date", inplace=True)
    actual.drop(columns=["Index"], inplace=True)
    actual.rename(columns={"Flow": "ActualFlow"}, inplace=True)

    mif = pd.read_csv(mif_csv, skiprows=6, names=["Index", "Date", "Flow"])
    mif["Date"] = pd.to_datetime(mif["Date"], format="%d%b%Y", errors="coerce")
    mif["Flow"] = pd.to_numeric(mif["Flow"], errors="coerce")
    mif = mif.dropna(subset=["Date", "Flow"])
    mif.set_index("Date", inplace=True)
    mif.drop(columns=["Index"], inplace=True)
    mif.rename(columns={"Flow": "MinFlow"}, inplace=True)

    df = pd.concat([actual, mif], axis=1)
    df["Difference"] = df["ActualFlow"] - df["MinFlow"]
    return df

def plot_flow_difference(df, title="Min Flow Difference", scenario_label=None):
    if scenario_label is None:
        scenario_label = FLOW_ANALYSIS_SCENARIO if FLOW_ANALYSIS_SCENARIO else "scenario"
    
    style = {"color": "red", "linestyle": ":", "label": f"{scenario_label} Adjusted Eflows", "linewidth": 1.5}

    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(df.index, df["Difference"], **style)
    ax.axhline(0, linestyle="--", color="gray", linewidth=1)
    ax.set_title(title, fontsize=14)
    ax.set_ylabel("Flow Difference (CFS)", fontsize=12)
    ax.set_xlabel("Date", fontsize=12)
    ax.grid(True, which='major', linestyle='--', linewidth=0.5)
    ax.legend(fontsize=11)
    plt.tight_layout()

    filename = title.replace(" ", "_").replace(":", "").lower() + ".png"
    output_path = os.path.join(os.getcwd(), filename)
    fig.savefig(output_path, dpi=DPI)
    print(f"Plot saved to: {output_path}")

    plt.show()

##### Parallel Coordinates Plots

In [19]:
print(list(combined_df.columns))

['All_Prob_S_SHSTA_flood', 'All_Prob_S_SHSTA_dead', 'Sept_Prob_S_SHSTA_flood', 'Sept_Prob_S_SHSTA_dead', 'All_Prob_S_OROVL_flood', 'All_Prob_S_OROVL_dead', 'Sept_Prob_S_OROVL_flood', 'Sept_Prob_S_OROVL_dead', 'All_Prob_S_TRNTY_flood', 'All_Prob_S_TRNTY_dead', 'Sept_Prob_S_TRNTY_flood', 'Sept_Prob_S_TRNTY_dead', 'All_Prob_S_SLUIS_CVP_flood', 'All_Prob_S_SLUIS_CVP_dead', 'Sept_Prob_S_SLUIS_CVP_flood', 'Sept_Prob_S_SLUIS_CVP_dead', 'All_Prob_S_SLUIS_SWP_flood', 'All_Prob_S_SLUIS_SWP_dead', 'Sept_Prob_S_SLUIS_SWP_flood', 'Sept_Prob_S_SLUIS_SWP_dead', 'All_Prob_S_MLRTN_flood', 'Sept_Prob_S_MLRTN_flood', 'All_Prob_S_MLRTN_dead', 'Sept_Prob_S_MLRTN_dead', 'All_Prob_S_MELON_flood', 'Sept_Prob_S_MELON_flood', 'All_Prob_S_MELON_dead', 'Sept_Prob_S_MELON_dead', 'Apr_Avg_S_RESTOT_NOD_TAF', 'Sep_Avg_S_RESTOT_NOD_TAF', 'Ann_Avg_S_RESTOT_NOD_TAF', 'Apr_Avg_S_RESTOT_sTAF', 'Sep_Avg_S_RESTOT_sTAF', 'Ann_Avg_S_RESTOT_sTAF', 'Apr_Avg_S_SHSTA_TAF', 'Sep_Avg_S_SHSTA_TAF', 'AprS_SHSTA__CV', 'SeptS_SHSTA__CV

In [20]:
# Auto-generate metric groups from column patterns

def build_metric_groups(df):
    """Generate metric groups by matching column name patterns."""
    cols = list(df.columns)
    
    def match(patterns, exclude=None):
        """Return columns matching any pattern, optionally excluding others."""
        result = []
        for c in cols:
            c_str = str(c)
            if any(p in c_str for p in patterns):
                if exclude is None or not any(e in c_str for e in exclude):
                    result.append(c)
        return result
    
    groups = {
        'G01_Floods': match(['Prob'], exclude=['dead']) + [c for c in cols if 'flood' in str(c).lower()],
        'G02_Deadpools': match(['Prob'], exclude=['flood']) + [c for c in cols if 'dead' in str(c).lower()],
        'G03_Storage_Apr': [c for c in cols if str(c).startswith('Apr') and ('S_' in str(c) or 'TAF' in str(c) or '_CV' in str(c))],
        'G04_Storage_Sep': [c for c in cols if (str(c).startswith('Sep') or str(c).startswith('Ann_Avg_S_')) and ('S_' in str(c) or 'TAF' in str(c) or '_CV' in str(c))],
        'G05_Deliveries_Flows': [c for c in cols if any(p in str(c) for p in ['DEL_', 'NDO_', 'D_TOTAL_'])],
        'G06_Salinity': [c for c in cols if any(p in str(c) for p in ['X2_', 'EC_MONTH', 'EM_EC', 'JP_EC'])]}
    
    for k in groups:
        groups[k] = list(dict.fromkeys(groups[k]))
    
    return groups

GROUPS = build_metric_groups(combined_df)
print(f"Generated {len(GROUPS)} metric groups with {sum(len(v) for v in GROUPS.values())} total columns")

Generated 6 metric groups with 101 total columns


##### Change from probability (0-1) to percent (0-100)

In [21]:
prob_like_cols = [c for c in combined_df.columns if c.startswith(('All_Prob_', 'Sept_Prob_'))]
if prob_like_cols:
    to_scale = [c for c in prob_like_cols if np.nanmax(np.abs(combined_df[c].values)) <= 1.01]
    print(f"Scaling {len(to_scale)} / {len(prob_like_cols)} probability columns to percent.")
    combined_df[to_scale] = combined_df[to_scale] * 100.0
else:
    print("No probability-like columns found to scale.")

plots_dir = os.path.join(metrics_root, "parallel_plots")
os.makedirs(plots_dir, exist_ok=True)

Scaling 28 / 28 probability columns to percent.


In [22]:
idx_lookup = {str(idx).strip().lower(): idx for idx in combined_df.index}
baseline_key = BASELINE_ID.lower()
if baseline_key in idx_lookup:
    BASELINE_ROW = idx_lookup[baseline_key]
else:
    matches = [idx for idx in combined_df.index if baseline_key in str(idx).lower()]
    if matches:
        BASELINE_ROW = matches[0]
    else:
        raise KeyError(f"Baseline '{BASELINE_ID}' not found in combined_df.index")

combined_df_pct = percent_change_from_baseline(combined_df, BASELINE_ROW)
scenario_ids = list(map(str, combined_df.index))

In [23]:
idx_like = [str(i).strip().lower() for i in combined_df.index]
def _resolve_label(hid):
    if hid in idx_like:
        return combined_df.index[idx_like.index(hid)]
    hits = [combined_df.index[i] for i, lab in enumerate(idx_like) if hid in lab]
    return hits[0] if hits else None

hl_idx = []
hl_colors = []
hl_labels = []
missing = []

for hid, color, label in highlight_config:
    resolved = _resolve_label(hid)
    if resolved is None:
        missing.append(hid)
        continue
    if resolved in hl_idx:
        continue
    hl_idx.append(resolved)
    hl_colors.append(color)
    hl_labels.append(label)

if missing:
    print(f"[WARN] highlight IDs not found in index and skipped: {missing}")

if BASELINE_ROW not in hl_idx:
    hl_idx = [BASELINE_ROW] + hl_idx
    hl_colors = ['#000000'] + hl_colors
    hl_labels = [baseline_label_text] + hl_labels
else:
    order = hl_idx.index(BASELINE_ROW)
    hl_idx = [hl_idx[order]] + hl_idx[:order] + hl_idx[order+1:]
    hl_colors = ['#000000'] + hl_colors[:order] + hl_colors[order+1:]
    hl_labels = [baseline_label_text] + hl_labels[:order] + hl_labels[order+1:]

In [24]:
for gname, gcols in GROUPS.items():
    cols = cu.cols_present(combined_df, gcols)
    if not cols:
        print(f"[SKIP] {gname}: no matching columns in combined_df.")
        continue

    out_norm = os.path.join(plots_dir, f"parallel_{gname}.png")
    pu.custom_parallel_coordinates_highlight_scenarios(objs=combined_df[cols], columns_axes=cols, axis_labels=cols, ideal_direction='top', minmaxs=['max'] * len(cols), color_dict_categorical={1: 'grey'}, highlight_indices=hl_idx, highlight_colors=hl_colors, highlight_descriptions=hl_labels, save_fig_filename=out_norm, title=f"Parallel Line Plot — {gname}", fontsize=10, figsize=(22, 8), dpi=DPI)

    out_pct = os.path.join(plots_dir, f"parallel_pct_{gname}.png")
    pu.custom_parallel_coordinates_highlight_scenarios_baseline_at_zero(objs=combined_df_pct[cols], columns_axes=cols, axis_labels=cols, highlight_indices=hl_idx, highlight_colors=hl_colors, highlight_descriptions=hl_labels, save_fig_filename=out_pct, title=f"Change from Baseline ({BASELINE_ID}) — {gname}", fontsize=10, figsize=(22, 8), dpi=DPI)

    print(f"Saved: {out_norm}")
    print(f"Saved: {out_pct}")

Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_G01_Floods.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_pct_G01_Floods.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_G02_Deadpools.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_pct_G02_Deadpools.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_G03_Storage_Apr.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_pct_G03_Storage_Apr.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_G04_Storage_Sep.png
Saved: ../../CalSim3_Model_Runs/Scenarios/Group_Data_Extraction/metrics_output/parallel_plots/parallel_pct_G04_Storage_Sep.png
Saved: ../../C